In [246]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split

from sklearn.multioutput import MultiOutputRegressor


In [247]:
# CSV 파일 경로
csv_file_path = '/Users/jojanghyun/Desktop/KHUDA/toy_project/interest_preprocessed.csv'

# CSV 파일을 데이터프레임으로 로드
df = pd.read_csv(csv_file_path)

print(df)



       Unnamed: 0      DATE  SEX  AGE  AREA  INCOME  SEOUL  GYEONGI  BUSAN  \
0               0  20221003    1    4    14       2      2        3      2   
1               1  20221003    1    4    14       3      3        3      2   
2               2  20221003    1    2     2       3      3        2      3   
3               3  20221003    1    2     2       2      2        2      1   
4               4  20221003    1    1    14       1      0        2      3   
...           ...       ...  ...  ...   ...     ...    ...      ...    ...   
10250       10965  20230731    1    4     1       1      2        2      3   
10251       10966  20230731    1    3     1       4      2        2      2   
10252       10967  20230731    1    3    11       2      3        2      4   
10253       10968  20230731    0    1     8       1      2        2      2   
10254       10970  20230731    1    4     1       2      2        2      2   

       CHUNGNAM  CHUNGBUK  GYEONGNAM  GYEONGBUK  JEONNAM  JEONB

In [248]:

# 지역 별 값을 포함하는 새로운 특징 열 생성
df['SUM_INTRST'] = df.apply(lambda row: [row['SEOUL'], row['GYEONGI'], row['BUSAN'],row['CHUNGNAM'], row['CHUNGBUK'], row['GYEONGNAM'],row['GYEONGBUK'], row['JEONNAM'],row['JEONBUK'], row['GANGWON'], row['JEJU']], axis=1)

# 결과 확인
print(df)


       Unnamed: 0      DATE  SEX  AGE  AREA  INCOME  SEOUL  GYEONGI  BUSAN  \
0               0  20221003    1    4    14       2      2        3      2   
1               1  20221003    1    4    14       3      3        3      2   
2               2  20221003    1    2     2       3      3        2      3   
3               3  20221003    1    2     2       2      2        2      1   
4               4  20221003    1    1    14       1      0        2      3   
...           ...       ...  ...  ...   ...     ...    ...      ...    ...   
10250       10965  20230731    1    4     1       1      2        2      3   
10251       10966  20230731    1    3     1       4      2        2      2   
10252       10967  20230731    1    3    11       2      3        2      4   
10253       10968  20230731    0    1     8       1      2        2      2   
10254       10970  20230731    1    4     1       2      2        2      2   

       CHUNGNAM  CHUNGBUK  GYEONGNAM  GYEONGBUK  JEONNAM  JEONB

In [249]:
#df.to_csv('new_dataset.csv', index=False) 

In [250]:
# 특징 열 선택 (성별, 나이, 거주지역, 수입)
X = df.iloc[:, 1:5] 

# 타겟 열 선택 (예측하려는 목표)
y = df['SUM_INTRST'].tolist()  

# 다중 레이블 데이터를 MultiLabelBinarizer를 사용하여 이진 배열로 변환
#mlb = MultiLabelBinarizer()
#y = mlb.fit_transform(y)  # y는 다중 레이블 데이터

In [251]:
# 데이터를 학습 및 테스트 세트로 분할 (예: 80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [252]:
from xgboost import XGBRegressor


# XGBoost 모델 정의 및 초기화
xgb_regressor = XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=3)

# MultiOutputRegressor 사용
multi_output_regressor = MultiOutputRegressor(xgb_regressor)

# 모델 훈련
multi_output_regressor.fit(X_train, y_train)

# 테스트 데이터를 사용하여 모델 평가
y_pred = multi_output_regressor.predict(X_test)


In [253]:

# 정확도 계산
# Mean Square Error
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)

print("mse:", mse)

# R2 score
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)

print("r2:", r2)

# Mean Absolute Error
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)

print("mae:", mae)


mse: 0.8665834418480999
r2: 0.017068991776766837
mae: 0.6857371415754174


In [254]:
# 새로운 데이터를 예측하기
# 기존의 1열 데이터와 같은 데이터를 새로운 데이터로 추가

new_data = np.array([[1,3,2,3]])

new_predict = multi_output_regressor.predict(new_data)

print("새로운 데이터에 대한 예측 결과:", new_predict)

새로운 데이터에 대한 예측 결과: [[2.5020564 2.165386  2.2496266 2.0149186 2.0570893 2.0256577 1.9206907
  2.2842793 2.033449  2.6132438 2.7418802]]


In [255]:
# 새로운 데이터의 MSE 구하기
new_mse = mean_squared_error(df.iloc[1,5:16].tolist(), new_predict[0])

print("new data mse:", new_mse)

new data mse: 0.4097791295495005


In [257]:

sex, age, area, income = map(int, input('성별(남자:0, 여자:1) 나이(20대:0, 30대:1, 40대:2, 50대:3, 60대:4), 거주지역(강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,울산광역시,인천광역시,전라남도,전라북도,제주도,충청남도(세종시 포함),충청북도), 월소득수준(1:~300,~500~700,4:700~)을 띄어쓰기로 구분하여 입력하시오').split())
new_data = np.array([[sex, age,area,income]])

new_predict = multi_output_regressor.predict(new_data)

print("새로운 데이터에 대한 예측 결과:", new_predict)


새로운 데이터에 대한 예측 결과: [[1.7560837 1.9245986 2.322931  1.8810484 1.9127337 1.8035202 1.7706655
  2.0981004 2.043581  2.6426587 2.819713 ]]


In [ ]:

new_mse = mean_squared_error(df.iloc[1,5:16].tolist(), new_predict[0])

print("new data mse:", new_mse)

new data mse: 0.5896156107090268


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split



# 탐색할 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 75, 25],  # 탐색할 n_estimators 후보값 [50, 100, 200] 에서 50/[50, 75, 25]에서 75
    'learning_rate': [0.05, 0.1, 0.15],  # 탐색할 learning_rate 후보값 [0.01, 0.1, 0.2] 에서 0.1/[0.05, 0.1, 0.15]에서 1 
    'max_depth': [3, 2, 1],  # 탐색할 max_depth 후보값 [3, 4, 5]에서 5/[3, 2, 1]에서 2
}

# GridSearchCV 객체 생성
grid_search = GridSearchCV(xgb_regressor, param_grid, cv=5, verbose=2, n_jobs=-1)

# Grid search 수행 (학습 데이터 X_train, y_train을 사용)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 점수 출력
print("최적 하이퍼파라미터:", grid_search.best_params_)
print("최적 점수:", grid_search.best_score_)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
최적 하이퍼파라미터: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 50}
최적 점수: 0.016477366739203005
